# Solving optimization problems with the kernel Trust-Region algorithm 

In this demo we want to demonstrate how to use the kernel TR algorithm to solve a 2D optimization problem. 
The problem we want to adress is taken from the pyMOR tutorial: Model order reduction for PDE-constrained optimization problems. We first import some standard python libraries. 

Make sure to install numpy, scipy, pymor, vkoga, mpmath, sklearn and pandas via pip. 


In [ ]:
from pymor.basic import *
import numpy as np
import math as m

We define the optimization problem as in Chapter 7.1 of the thesis. The 'diameter' arguments defines the width of the mesh. In all of our numerical experiments we used 'diameter = 1/100'.Note that the meshwidth has a tremendous effect on the runtime.


In [ ]:
import importlib 
problems = importlib.import_module('2D_PDE_optimization_problem.problems')

problem = problems.linear_problem()
mu_bar = problem.parameters.parse([np.pi/2,np.pi/2])
fom, data = discretize_stationary_cg(problem, diameter=1/100, mu_energy_product=mu_bar)
parameter_space = fom.parameters.space(0, np.pi)

Before we start solving the optimization problem, we first visualize the objective function as a 3D plot. This is Figure 4 in the thesis. Take care: This computation takes some time, as it requires solving the FOM 2500 times. If you want to speed up the computation, define the linear problem again using a coarser mesh (i.e 'diameter = 1/25').

In [ ]:
import importlib 
contour = importlib.import_module('2D_PDE_optimization_problem.contour_plot_objective_function')
#from contour_plot_objective_function import plot_3d_surface, fom_objective_functional

ranges = parameter_space.ranges['diffusion']
XX = np.linspace(ranges[0] + 0.05, ranges[1], 50)
contour.plot_3d_surface(fom, contour.fom_objective_functional, XX, XX)

In the next step, we want to solve this optimization problem with a state of the art optimization algorithm, namely with the BFGS algorithm. We decribe the difference between gradient and no gradient in Chapter 7.2. To compare the results of the BFGS with the kernel TR algorithm, we listed the amount of FOMS if no gradient was used in the thesis (Table 3).

In [ ]:
bfgs = importlib.import_module('2D_PDE_optimization_problem.2D_bfgs_algorithm')

#rom 2D_bfgs_algorithm import fom_objective_functional, fom_gradient_of_functional, optimize_BFGS, report

amount_of_iters = 1
ranges = parameter_space.ranges['diffusion']

#Does not use gradients
data = bfgs.optimize_BFGS(bfgs.fom_objective_functional, ranges, amount_of_iters, fom)
bfgs.report(data, amount_of_iters)

#Uses gradients
#data = bfgs.optimize_BFGS(bfgs.fom_objective_functional, ranges, amount_of_iters, fom, bfgs.fom_gradient_of_functional)
#bfgs.report(data, amount_of_iters)
#np.save('reference_mu.npy', data['mu'])

We will now utilize the proposed kernel TR algorithm to solve the optimization problem again. Before we compare the results with the BFGS algorithm from above, we compare the TRs of the standard and the advanved formulation. The outcome is Figure 5 in the thesis. Note that these computations also require much more computational time, as we need to evaluate the FOM more often to get information about the TR and plot them. 

In [ ]:
tr_kernel = importlib.import_module('2D_PDE_optimization_problem.trust_region_kernel_algorithm')
from vkoga.kernels import Gaussian, IMQ, Wendland, Matern

ep=1
global_counter = 0
mu_k = [0.25, 0.5]
mu_k = problem.parameters.parse(mu_k)

kernel = IMQ(ep=ep)
RKHS_norm, _ , _ = tr_kernel.compute_RKHS_norm(kernel, fom, parameter_space)

TR_parameters={'radius': 0.5, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100,'starting_parameter': mu_k, 
               'max_iterations_armijo': 40, 'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
               'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8, 'kernel_width': ep, 'advanced': False, 'draw_TR': True}

mu_list, _ , list_delta = tr_kernel.TR_Kernel(fom, kernel, parameter_space, global_counter, RKHS_norm, TR_parameters)
tr_kernel.draw_TR_standard(list_delta, mu_list)

In [ ]:
tr_kernel = importlib.import_module('2D_PDE_optimization_problem.trust_region_kernel_algorithm')

global_counter = 0

TR_parameters={'radius': 2, 'sub_tolerance': 1e-5, 'max_iterations': 10, 'max_iterations_subproblem': 100,'starting_parameter': mu_k, 
               'max_iterations_armijo': 40, 'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
               'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8, 'kernel_width': ep, 'advanced': True, 'draw_TR': True}

mu_list,TR_plot_matrix , _ = tr_kernel.TR_Kernel(fom, kernel, parameter_space, global_counter, RKHS_norm, TR_parameters)
tr_kernel.draw_TR_advanced(TR_plot_matrix, mu_list)

We now utilize the kernel TR algorithm to solve the 2D optimization problem. This reproduces Tables 5-8. We start with the standard formulation of the IMQ kernel. 

We need to specify all of the parameters in the dictionary TR_parameters. If they are missing, a default value is choosen. The defaults can be found at the beginning of the TR_Kernel method. 

We also need to specify the kernel name. Currently supported: 'gauss', imq', 'mat2' and 'wen{}2' (set the dimesion of the wendland kernel in the {}).

In [ ]:

tr_kernel = importlib.import_module('2D_PDE_optimization_problem.trust_region_kernel_algorithm')

amount_of_iters = 10

gamma_list = [0.8, 0.9, 1, 1.1, 1.2]
kernel_name = 'imq'

TR_parameters={'radius': 0.5, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100, 'max_iterations_armijo': 40, 
            'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
            'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8, 'advanced': False, 'draw_TR': False}

data = tr_kernel.optimize_PDE(fom, parameter_space, tr_kernel.TR_Kernel, kernel_name, gamma_list, TR_parameters, amount_of_iters)

tr_kernel.report_kernel_TR(data, gamma_list, amount_of_iters)

Continue with the advanced formulation of the IMQ kernel.

In [ ]:

gamma_list = [0.8, 0.9, 1, 1.1, 1.2]
kernel_name = 'imq'

TR_parameters={'radius': 2, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100, 'max_iterations_armijo': 40, 
            'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
            'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8,  'advanced': True, 'draw_TR': False}

data = tr_kernel.optimize_PDE(fom, parameter_space, tr_kernel.TR_Kernel, kernel_name, gamma_list, TR_parameters, amount_of_iters)

In [ ]:
tr_kernel.report_kernel_TR(data, gamma_list, amount_of_iters)

We change the kernel to the Gaussian and begin with the standard formulation.

In [ ]:
#Standard formulation of the Gaussian kernel. 
gamma_list = [0.9, 0.95, 1, 1.05, 1.1]
kernel_name = 'gauss'

TR_parameters={'radius': 0.5, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100, 'max_iterations_armijo': 40, 
            'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
            'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8,  'advanced': False, 'draw_TR': False}

data = tr_kernel.optimize_PDE(fom, parameter_space, tr_kernel.TR_Kernel, kernel_name, gamma_list, TR_parameters, amount_of_iters)

In [ ]:
tr_kernel.report_kernel_TR(data, gamma_list, amount_of_iters)

The last computation is the Gaussian kernel with the advanced formulation.

In [ ]:

gamma_list = [0.9, 0.95, 1, 1.05, 1.1]
kernel_name = 'gauss'

TR_parameters={'radius': 2, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100, 'max_iterations_armijo': 40, 
            'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
            'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8,  'advanced': True, 'draw_TR': False}

data = tr_kernel.optimize_PDE(fom, parameter_space, tr_kernel.TR_Kernel, kernel_name, gamma_list, TR_parameters, amount_of_iters)

In [ ]:
tr_kernel.report_kernel_TR(data, gamma_list, amount_of_iters)